In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [ ]:
!kaggle datasets download -d umongsain/robovox-sp-cup-2024

100% 7.08G/7.08G [06:20<00:00, 17.0MB/s]
100% 7.08G/7.08G [06:20<00:00, 20.0MB/s]


In [ ]:
!unzip "/content/robovox-sp-cup-2024.zip" -d "/content"


Streaming output truncated to the last 5000 lines.
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/862c02d845.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/863286eb3f.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/863b206fde.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/863c401628.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/86410f1d77.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/864145bb4c.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/86421b4fea.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/8644ce56b3.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/864991f2d1.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/8651378821.wav  
  inflating: /content/ROBOVOX_SP_CUP_2024/data/single-channel/test/86548f38e1.wav  
  inflating: /content/ROB

In [ ]:
os.chdir("/content")

In [ ]:
## Install NeMo

# Install dependencies
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install text-unidecode

BRANCH = 'r1.22.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]
# Install TorchAudio
!pip install torchaudio>=0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=42b46460cf912f531a479df0cd2d7b339bf340c642eef4f55c9e288684042e64
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1
Suggested packages:
  libsox-fmt-all
The following NEW packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa libsox-fmt-base libsox3 libwavpack1 sox
0 upgraded, 7 newly installed, 0 to remove and 29 not upgraded.
Need to get 617 kB of archives.
After this operation, 1,764 kB of additi

In [ ]:

NEMO_ROOT = os.getcwd()
print(NEMO_ROOT)

/content


In [ ]:
import os
import shutil

def rearrange_files(input_folder, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through each file in the input folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)

        # Check if the path is a file (not a subdirectory)
        if os.path.isfile(file_path):
            # Extract the prefix before the first underscore
            prefix = filename.split('-')[0]

            # Create a subfolder in the output folder based on the prefix
            subfolder_path = os.path.join(output_folder, prefix)
            if not os.path.exists(subfolder_path):
                os.makedirs(subfolder_path)

            # Move the file to the corresponding subfolder
            shutil.move(file_path, os.path.join(subfolder_path, filename))

if __name__ == "__main__":
    # Replace 'input_folder' and 'output_folder' with your actual folder paths
    input_folder = "/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment"
    output_folder = "/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper"

    rearrange_files(input_folder, output_folder)


In [ ]:
data_dir = "/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper"
!find {data_dir}  -iname "*.wav" > "/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper/train_all.txt"

In [ ]:
!head -n 3 {data_dir}/train_all.txt

/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper/spk_32/spk_32-32_39_1_0_d1_ch5.wav
/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper/spk_32/spk_32-32_38_1_1_d5_ch5.wav
/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper/spk_32/spk_32-32_38_1_0_d6_ch5.wav


In [ ]:
!pip install sox

In [ ]:
import torch
import pytorch_lightning as pl

In [ ]:
import sox
if not os.path.exists('scripts'):
  print("Downloading necessary scripts")
  !mkdir -p scripts/speaker_tasks
  !wget -P scripts/speaker_tasks/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/scripts/speaker_tasks/filelist_to_manifest.py
!python {NEMO_ROOT}/scripts/speaker_tasks/filelist_to_manifest.py --filelist {data_dir}/train_all.txt --id 6 --out {data_dir}/all_manifest.json

--2024-01-19 11:28:21--  https://raw.githubusercontent.com/NVIDIA/NeMo/r1.22.0/scripts/speaker_tasks/filelist_to_manifest.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8647 (8.4K) [text/plain]
Saving to: ‘scripts/speaker_tasks/filelist_to_manifest.py’

filelist_to_manifes 100%[===================>]   8.44K  --.-KB/s    in 0s      

2024-01-19 11:28:22 (78.5 MB/s) - ‘scripts/speaker_tasks/filelist_to_manifest.py’ saved [8647/8647]

[NeMo W 2024-01-19 11:28:30 transformer_bpe_models:59] Could not import NeMo NLP collection which is required for speech translation model.
100% 225/225 [00:00<00:00, 390.73it/s]
wrote /content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper/train_all_manifest.json
100% 225/225 [00:00<00:00, 23597.09it/s]
wrote /cont

In [ ]:
!find {data_dir} -iname "*.wav" > {data_dir}/test_all.txt
!python {NEMO_ROOT}/scripts/speaker_tasks/filelist_to_manifest.py --filelist {data_dir}/test_all.txt --id -6 --out {data_dir}/test.json

[NeMo W 2024-01-18 14:57:19 transformer_bpe_models:59] Could not import NeMo NLP collection which is required for speech translation model.
100% 225/225 [00:00<00:00, 744.14it/s]
wrote /content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper/test_all_manifest.json
100% 225/225 [00:00<00:00, 56912.22it/s]
wrote /content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper/test.json


In [ ]:
train_manifest = os.path.join(data_dir,'train_all_manifest.json')
validation_manifest = os.path.join(data_dir,'dev.json')
test_manifest = os.path.join(data_dir,'dev.json')

In [ ]:
import nemo
# NeMo's ASR collection - This collection contains complete ASR models and
# building blocks (modules) for ASR
import nemo.collections.asr as nemo_asr
from omegaconf import OmegaConf

[NeMo W 2024-01-19 11:12:59 transformer_bpe_models:59] Could not import NeMo NLP collection which is required for speech translation model.


In [ ]:
# This line will print the entire config of sample TitaNet model
!mkdir conf
!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/speaker_tasks/recognition/conf/titanet-large.yaml
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/titanet-large.yaml')
config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(config))

--2024-01-18 14:57:23--  https://raw.githubusercontent.com/NVIDIA/NeMo/r1.22.0/examples/speaker_tasks/recognition/conf/titanet-large.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4483 (4.4K) [text/plain]
Saving to: ‘conf/titanet-large.yaml’

titanet-large.yaml  100%[===================>]   4.38K  --.-KB/s    in 0s      

2024-01-18 14:57:23 (62.7 MB/s) - ‘conf/titanet-large.yaml’ saved [4483/4483]

name: TitaNet-L
sample_rate: 16000
model:
  train_ds:
    manifest_filepath: ???
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: null
        prob: 0.5
        min_snr_db: 0
        max_snr_

In [ ]:
config.model.train_ds.manifest_filepath = train_manifest
#config.model.validation_ds.manifest_filepath = validation_manifest

In [ ]:
config.model.decoder.num_classes = 75

In [ ]:
!wget -P conf https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/speaker_tasks/recognition/conf/titanet-finetune.yaml
MODEL_CONFIG = os.path.join(NEMO_ROOT,'conf/titanet-finetune.yaml')
finetune_config = OmegaConf.load(MODEL_CONFIG)
print(OmegaConf.to_yaml(finetune_config))

--2024-01-18 14:57:23--  https://raw.githubusercontent.com/NVIDIA/NeMo/r1.22.0/examples/speaker_tasks/recognition/conf/titanet-finetune.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4507 (4.4K) [text/plain]
Saving to: ‘conf/titanet-finetune.yaml’

titanet-finetune.ya 100%[===================>]   4.40K  --.-KB/s    in 0s      

2024-01-18 14:57:23 (65.4 MB/s) - ‘conf/titanet-finetune.yaml’ saved [4507/4507]

name: TitaNet-Finetune
sample_rate: 16000
init_from_pretrained_model:
  speaker_tasks:
    name: titanet_large
    include:
    - preprocessor
    - encoder
    exclude:
    - decoder.final
model:
  train_ds:
    manifest_filepath: ???
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio

In [ ]:
test_manifest = os.path.join(data_dir,'test.json')
finetune_config.model.train_ds.manifest_filepath = train_manifest
finetune_config.model.train_ds.batch_size = 9
finetune_config.model.validation_ds.manifest_filepath = train_manifest
finetune_config.model.validation_ds.batch_size = 9
finetune_config.model.decoder.num_classes = 75

In [ ]:
# Setup the new trainer object
# Let us modify some trainer configs for this demo
# Checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'

trainer_config = OmegaConf.create(dict(

    devices=1,
    accelerator=accelerator,
    max_epochs=5,
    max_steps=-1,  # computed at runtime if not set
    num_nodes=1,
    accumulate_grad_batches=1,
    enable_checkpointing=False,  # Provided by exp_manager
    logger=False,  # Provided by exp_manager
    log_every_n_steps=1,  # Interval of logging.
    val_check_interval=1.0,  # Set to 0.25 to check 4 times per epoch, or an int for number of iterations
))
print(OmegaConf.to_yaml(trainer_config))

devices: 1
accelerator: gpu
max_epochs: 5
max_steps: -1
num_nodes: 1
accumulate_grad_batches: 1
enable_checkpointing: false
logger: false
log_every_n_steps: 1
val_check_interval: 1.0



In [ ]:
trainer_finetune = pl.Trainer(**trainer_config)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


In [ ]:
from nemo.utils.exp_manager import exp_manager
log_dir_finetune = exp_manager(trainer_finetune, config.get("exp_manager", None))
print(log_dir_finetune)

[NeMo I 2024-01-18 14:57:24 exp_manager:394] Experiments will be logged at /content/nemo_experiments/TitaNet-L/2024-01-18_14-57-24
[NeMo I 2024-01-18 14:57:24 exp_manager:835] TensorboardLogger has been set up
/content/nemo_experiments/TitaNet-L/2024-01-18_14-57-24


In [ ]:
speaker_model = nemo_asr.models.EncDecSpeakerLabelModel(cfg=finetune_config.model, trainer=trainer_finetune)
speaker_model.maybe_init_from_pretrained_checkpoint(finetune_config)

[NeMo I 2024-01-18 14:57:24 collections:445] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-01-18 14:57:24 collections:446] Dataset loaded with 225 items, total duration of  0.41 hours.
[NeMo I 2024-01-18 14:57:24 collections:448] # 225 files loaded accounting to # 75 labels


[NeMo W 2024-01-18 14:57:24 label_models:189] Total number of 75 found in all the manifest files.


[NeMo I 2024-01-18 14:57:24 collections:445] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-01-18 14:57:24 collections:446] Dataset loaded with 225 items, total duration of  0.41 hours.
[NeMo I 2024-01-18 14:57:24 collections:448] # 225 files loaded accounting to # 75 labels
[NeMo I 2024-01-18 14:57:24 collections:445] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2024-01-18 14:57:24 collections:446] Dataset loaded with 225 items, total duration of  0.41 hours.
[NeMo I 2024-01-18 14:57:24 collections:448] # 225 files loaded accounting to # 75 labels
[NeMo I 2024-01-18 14:57:24 features:289] PADDING: 16
[NeMo I 2024-01-18 14:57:25 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/titanet_large/versions/v1/files/titanet-l.nemo to /root/.cache/torch/NeMo/NeMo_1.22.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2024-01-18 14:57:27 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2024-01-18 14:57:27 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2024-01-18 14:57:27 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2024-01-18 14:57:27 features:289] PADDING: 16
[NeMo I 2024-01-18 14:57:28 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.22.0/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2024-01-18 14:57:28 modelPT:1150] Model checkpoint partially restored from pretrained checkpoint with name `titanet_large`
[NeMo I 2024-01-18 14:57:28 modelPT:1152] The following parameters were excluded when loading from pretrained checkpoint with name `titanet_large` : ['decoder.final.weight']
[NeMo I 2024-01-18 14:57:28 modelPT:1155] Make sure that this is what you wanted!


In [ ]:
!nvidia-smi

Thu Jan 18 14:57:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
## Fine-tuning for 5 epochs¶
trainer_finetune.fit(speaker_model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-01-18 14:57:35 modelPT:728] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.0001
        maximize: False
        weight_decay: 0.0002
    
    Parameter Group 1
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 0.001
        maximize: False
        weight_decay: 0.0002
    )
[NeMo I 2024-01-18 14:57:35 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f8e93faaec0>" 
    will be used during training (effective maximum steps = 125) - 
    Parameters : 
    (warmup_ratio: 0.1
    min_lr: 0.0
    max_steps: 125
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type                              | Params
----------------------------------------------------------------------
0 | loss            | AngularSoftmaxLoss                | 0     
1 | eval_loss       | AngularSoftmaxLoss                | 0     
2 | _accuracy       | TopKClassificationAccuracy        | 0     
3 | preprocessor    | AudioToMelSpectrogramPreprocessor | 0     
4 | encoder         | ConvASREncoder                    | 19.4 M
5 | decoder         | SpeakerDecoder                    | 2.8 M 
6 | _macro_accuracy | MulticlassAccuracy                | 0     
----------------------------------------------------------------------
22.1 M    Trainable params
0         Non-trainable params
22.1 M    Total params
88.547    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 75: 'val_loss' reached 9.60382 (best 9.60382), saving model to '/content/nemo_experiments/TitaNet-L/2024-01-18_14-57-24/checkpoints/TitaNet-L--val_loss=9.6038-epoch=2.ckpt' as top 3


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 100: 'val_loss' reached 8.83222 (best 8.83222), saving model to '/content/nemo_experiments/TitaNet-L/2024-01-18_14-57-24/checkpoints/TitaNet-L--val_loss=8.8322-epoch=3.ckpt' as top 3


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 125: 'val_loss' reached 8.37403 (best 8.37403), saving model to '/content/nemo_experiments/TitaNet-L/2024-01-18_14-57-24/checkpoints/TitaNet-L--val_loss=8.3740-epoch=4.ckpt' as top 3
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
final_checkpoint = "/content/drive/MyDrive/TitaNet-L--val_loss=7.6501-epoch=5-last.ckpt"
restored_model = nemo_asr.models.EncDecSpeakerLabelModel.load_from_checkpoint(final_checkpoint)

[NeMo W 2024-01-19 11:13:14 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper/train_all_manifest.json
    sample_rate: 16000
    labels:
    - spk_10
    - spk_11
    - spk_12
    - spk_13
    - spk_14
    - spk_15
    - spk_16
    - spk_17
    - spk_18
    - spk_19
    - spk_2
    - spk_20
    - spk_21
    - spk_22
    - spk_23
    - spk_24
    - spk_25
    - spk_26
    - spk_27
    - spk_28
    - spk_29
    - spk_3
    - spk_30
    - spk_31
    - spk_32
    - spk_33
    - spk_34
    - spk_35
    - spk_36
    - spk_37
    - spk_38
    - spk_39
    - spk_4
    - spk_40
    - spk_41
    - spk_42
    - spk_43
    - spk_44
    - spk_45
    - spk_46
    - spk_47
    - spk_48
    - spk_49
    - spk_5
    - spk_50
    - spk_51
    - spk_52
    - spk_53
 

[NeMo I 2024-01-19 11:13:14 features:289] PADDING: 16


In [ ]:
import pandas as pd
import torch
import os
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv("/content/ROBOVOX_SP_CUP_2024/data/single-channel/signle-channel-trials.trl", sep="\t", header=None)

In [ ]:
def undo_rearrange_files(input_folder, output_folder):
    # Iterate through each subfolder in the output folder
    for subfolder_name in os.listdir(output_folder):
        subfolder_path = os.path.join(output_folder, subfolder_name)

        # Check if the path is a subfolder
        if os.path.isdir(subfolder_path):
            # Iterate through each file in the subfolder
            for filename in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, filename)

                # Move the file back to the input folder
                shutil.move(file_path, os.path.join(input_folder, filename))

            # Remove the now-empty subfolder
            os.rmdir(subfolder_path)

if __name__ == "__main__":
    # Replace 'input_folder' and 'output_folder' with your actual folder paths
    input_folder = "/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment"
    output_folder = "/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment_proper"

    undo_rearrange_files(input_folder, output_folder)

In [ ]:
enrollment_path = "/content/ROBOVOX_SP_CUP_2024/data/single-channel/enrollment"

In [ ]:
file_list = [f for f in Path(enrollment_path).rglob('*') if f.is_file()]
speaker_to_file_map = dict()

for enrollment in file_list:
  speaker = enrollment.name.split('-')[0]
  array = speaker_to_file_map.get(speaker, -1)

  if(array==-1):
    speaker_to_file_map[speaker] = [str(enrollment)]
  else:
    speaker_to_file_map[speaker].append(str(enrollment))

In [ ]:
embeddings = {}

for speaker, file_paths in speaker_to_file_map.items():
    # Process enrollments for each speaker (replace this with your actual processing logic)
    enrollement_embedding = [restored_model.get_embedding(file_path) for file_path in file_paths]

    # Store the processed data in the new dictionary
    embeddings[speaker] = enrollement_embedding

In [ ]:
enrollment_list1 = speaker_to_file_map.get("spk_72")
enrollment_list2 = speaker_to_file_map.get("spk_71")
restored_model.verify_speakers(enrollment_list1[0],enrollment_list2[1])

[NeMo I 2024-01-19 11:13:57 label_models:585]  two audio files are from different speakers


False

In [ ]:
# After changing the range of the similarity to [0, 1]

with open('output.txt', 'w') as output:
  for i in range(len(df)):
    speaker, test_file = df.iloc[i][0], df.iloc[i][1]

    test_embedding = restored_model.get_embedding(f"/content/ROBOVOX_SP_CUP_2024/data/single-channel/test/{test_file}.wav").squeeze()

    # enrollment_list = speaker_to_file_map.get(speaker)

    # e_embedding_1 = restored_model.get_embedding(enrollment_list[0])
    # e_embedding_2 = restored_model.get_embedding(enrollment_list[1])
    # e_embedding_3 = restored_model.get_embedding(enrollment_list[2])

    embedding_list = embeddings.get(speaker)

    average_embedding = torch.mean(torch.stack(embedding_list, dim=0), dim=0).squeeze()

    test_embedding_normalized = test_embedding / torch.linalg.norm(test_embedding.cpu())
    average_embedding_normalized = average_embedding / torch.linalg.norm(average_embedding.cpu())

    similarity = torch.dot(test_embedding_normalized, average_embedding_normalized) / ((torch.dot(test_embedding_normalized, test_embedding_normalized) * torch.dot(average_embedding_normalized, average_embedding_normalized)) ** 0.5)
    similarity = (similarity + 1) / 2

    # similarity = cosine_similarity(test_embedding.cpu(), average_embedding.cpu())[0,0]

    output.write(f"{speaker}\t{test_file}\t{similarity}\n")

KeyboardInterrupt: 